In [1]:
import pandas as pd
import numpy as np
import time
import mlflow

import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

# Exploring data

In [ ]:
dataPath = 'nlp-getting-started/train.csv'
data = pd.read_csv(dataPath, index_col=0)

In [ ]:
data.info()

### Count

In [ ]:
data.count()

### Nan

In [ ]:
data.isna().sum()

In [ ]:
dataNonNull = data.dropna()

In [ ]:
print(f'{"Columns":20}: {"All":10} {"NonNull":10} {"%NonNull":10} {"Difference"}')
for idx, col in enumerate(data.columns):
    allValue = data.count()[idx]
    nonNullValue = dataNonNull.count()[idx]
    per = nonNullValue*100/allValue
    diff = allValue - nonNullValue
    print(f'{col:20}: {allValue} {nonNullValue:10} {np.round(per):10} {diff:10}')

In [ ]:
data['keyword'].fillna("", inplace=True)
data['location'].fillna("", inplace=True)

## Target

In [ ]:
columnName = 'target'

#----------------------
def getCategoricalColumn(value):
    if value == 1: return "Disaster"
    else: return "Not disaster"
    
CategoricalColumn = data[columnName].apply(getCategoricalColumn)
CategoricalColumn.name = 'catTarget'

df = pd.concat([data, CategoricalColumn], axis=1)
#----------------------

groups = []
for group, subset in df.groupby(by=CategoricalColumn.name):
    groups.append({
        CategoricalColumn.name: group,
        'Count': len(subset)
    })

lenData = data[columnName].count()

dataCategoricalQuality = pd.DataFrame(groups)

fig, ax = plt.subplots(figsize=(4, 4))

dataCategoricalQuality.plot.bar(x=CategoricalColumn.name, ax=ax)

for i in range(len(groups)):
    value = str(groups[i]['Count'])+': '+str(np.round(groups[i]['Count']*100/lenData))+'%'
    ax.text(i, groups[i]['Count'], value , horizontalalignment='center', 
            verticalalignment='bottom')

ax.set_ylim(0, lenData - lenData/5)

ax.set_xlabel('target')
ax.set_ylabel('Count')
ax.set_title('Sum: '+ str(lenData) )
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

### target vs keyword

In [ ]:
data['keyword'] = data['keyword'].str.replace('%20', ' ')

In [ ]:
columnNameA = 'target'
columnNameB = 'keyword'

crossTable = pd.crosstab(index=data[columnNameB],
                         columns=data[columnNameA],
                         margins=True)

crossTable.rename(columns={0 : 'Not disaster',1 : 'Disaster',}, inplace=True)

In [ ]:
print('Most frequent Keywords for Disaster')
crossTable.sort_values(by='Disaster', ascending=False).head(10)

In [ ]:
print('Most frequent Keywords for Not disaster')
crossTable.sort_values(by='Not disaster', ascending=False).head(10)

#### target vs keyword length (character)

In [ ]:
data['keywordLengthChar'] = data['keyword'].apply(len)

In [ ]:
columnNameA = 'target'
columnNameB = 'keywordLengthChar'

sns.boxplot(data=data, x=columnNameA, y=columnNameB)


plt.tight_layout()
plt.show()

In [ ]:
columnNameA = 'target'
columnNameB = 'keywordLengthChar'

g = sns.FacetGrid(data, col=columnNameA)
g.map(sns.histplot, columnNameB, bins=30)

plt.tight_layout()
plt.show()

We observe that keyword lengths are longer in Not Disaster than in Disaster

### target vs location

In [ ]:
columnNameA = 'target'
columnNameB = 'location'

crossTable = pd.crosstab(index=data[columnNameB],
                         columns=data[columnNameA],
                         margins=True)

crossTable.rename(columns={0 : 'Not disaster',1 : 'Disaster',}, inplace=True)

In [ ]:
print('Most frequent Locations for Disaster')
crossTable.sort_values(by='Disaster', ascending=False).head(10)

In [ ]:
print('Most frequent Locations for Not disaster')
crossTable.sort_values(by='Not disaster', ascending=False).head(10)

### target vs text

#### Number of character (including space)

In [ ]:
data['textLengthChar'] = data['text'].apply(len)

In [ ]:
columnName = 'textLengthChar'

ax = (data[columnName]).plot.box(figsize=(3, 4))
ax.set_ylabel(columnName)

plt.tight_layout()
plt.show()

In [ ]:
columnNameA = 'target'
columnNameB = 'textLengthChar'

g = sns.FacetGrid(data, col=columnNameA)
g.map(sns.histplot, columnNameB, bins=30)

plt.tight_layout()
plt.show()

We observe that text lengths are longer in Not Disaster than in Disaster

In [ ]:
columnNameA = 'target'
columnNameB = 'textLengthChar'

sns.boxplot(data=data, x=columnNameA, y=columnNameB)

plt.tight_layout()
plt.show()

#### Number of words

In [ ]:
def getWordTextLength(text):
    return len(text.split())
data['textLengthWord'] = data['text'].apply(getWordTextLength)

In [ ]:
columnName = 'textLengthWord'

ax = (data[columnName]).plot.box(figsize=(3, 4))
ax.set_ylabel(columnName)

plt.tight_layout()
plt.show()

In [ ]:
columnNameA = 'target'
columnNameB = 'textLengthWord'

g = sns.FacetGrid(data, col=columnNameA)
g.map(sns.histplot, columnNameB, bins=30)

plt.tight_layout()
plt.show()

We observe that number of words in text are longer in Not Disaster than in Disaster

In [ ]:
columnNameA = 'target'
columnNameB = 'textLengthWord'

sns.boxplot(data=data, x=columnNameA, y=columnNameB)

plt.tight_layout()
plt.show()

#### Link in text

In [ ]:
pattern = r'(https?://\S+)'
data['link']= data["text"].str.extract(pattern)
    
data['containLink'] = data['link'].notna()

In [ ]:
columnNameA = 'target'
columnNameB = 'containLink'

crossTable = pd.crosstab(index=data[columnNameB],
                         columns=data[columnNameA],
                         margins=True)

crossTable.rename(columns={0 : 'Not disaster',1 : 'Disaster',}, inplace=True)
crossTable.rename(index={False : 'No link',True : 'Link',}, inplace=True)

crossTable['Not disaster %'] = crossTable['Not disaster'] * 100 / crossTable['All']
crossTable['Disaster %'] = crossTable['Disaster'] * 100 / crossTable['All']

crossTable

We observe that there are more links in Disaster than Not Disaster.

# Processing data using PyPark

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext(master = "local", appName = "App").getOrCreate()
spark = SparkSession.builder.getOrCreate()

print(sc, sc.version, spark)

<SparkContext master=local appName=App> 3.2.0 <pyspark.sql.session.SparkSession object at 0x7f824179bc40>


## Loading data

In [4]:
trainPath = 'nlp-getting-started/train.csv'
testPath = 'nlp-getting-started/test.csv'

trainData = spark.read.format('csv').options(header='true', inferSchema='true', multiLine=True).load(trainPath)
testData = spark.read.format('csv').options(header='true', inferSchema='true', multiLine=True).load(testPath)

print('Number of row in Training:', trainData.count())
print('Number of row in Test:    ', testData.count())

Number of row in Training: 7613
Number of row in Test:     3263


## Preprocessing

In [142]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols
from pyspark.ml import Pipeline 
import pyspark.sql.functions as F
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, Word2Vec, StringIndexer, VectorAssembler, RobustScaler

In [195]:
class FillNanTransformer(Transformer, HasInputCol):
    @keyword_only
    def __init__(self, inputCol=None):
        super(FillNanTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, dataset):
        dataset = dataset.na.fill(value="",subset=[self.getInputCol()])
        return dataset
    
class TextNoLinkTransformer(Transformer, HasInputCol, HasOutputCol):
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(TextNoLinkTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, dataset):
        pattern = r'(https?://\S+)'
        dataset = dataset.withColumn(self.getOutputCol(), F.regexp_replace(F.col(self.getInputCol()), pattern, ""))
        return dataset
    
class ContainLinkTransformer(Transformer, HasInputCol, HasOutputCol):
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(ContainLinkTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, dataset):
        pattern = r'(https?://\S+)'
        dataset = dataset.withColumn(self.getOutputCol(), F.when(F.col(self.getInputCol()).rlike(pattern),1).otherwise(0))
        return dataset
    
class KeywordLengthTransformer(Transformer, HasInputCol, HasOutputCol):
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(KeywordLengthTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, dataset):
        dataset = dataset.withColumn(self.getOutputCol(), F.length(self.getInputCol()))
        return dataset
    
class TextNoLinkLengthTransformer(Transformer, HasInputCol, HasOutputCol):
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(TextNoLinkLengthTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, dataset):
        dataset = dataset.withColumn(self.getOutputCol(), F.length(self.getInputCol()))
        return dataset
    
class ConcatenateTransformer(Transformer, HasInputCols, HasOutputCol):
    @keyword_only
    def __init__(self, inputCols=None, outputCol=None):
        super(ConcatenateTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCols=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, dataset):
        dataset = dataset.withColumn(self.getOutputCol(), F.col(self.getInputCols()[0]))
        for colName in self.getInputCols()[1:]:
            dataset = dataset.withColumn(self.getOutputCol(), 
                F.concat_ws('@', F.col(self.getOutputCol()), F.col(colName)))
        return dataset

In [393]:
textFillNanTransformer = FillNanTransformer(inputCol="text")
keywordFillNanTransformer = FillNanTransformer(inputCol="keyword")
locationFillNanTransformer = FillNanTransformer(inputCol="location")

keywordIndexer = StringIndexer(inputCol="keyword", outputCol="keywordIndex", handleInvalid="keep")
locationIndexer = StringIndexer(inputCol="location", outputCol="locationIndex", handleInvalid="keep")

containLinkTransformer = ContainLinkTransformer(inputCol="text", outputCol="containLink")
textNoLinkTransformer = TextNoLinkTransformer(inputCol="text", outputCol="textNoLink")

regexTokenizer = RegexTokenizer(inputCol="textNoLink", outputCol="wordText", pattern="\\W")
stopWordsRemover = StopWordsRemover(inputCol="wordText", outputCol="wordTextNoSW")
word2Vec = Word2Vec(vectorSize=50, windowSize=10, minCount=0, inputCol="wordTextNoSW", outputCol="vecText")

concatenateTransformer = ConcatenateTransformer(inputCols=["keyword", "location", "textNoLink"], outputCol="concatText")
concatTextRegexTokenizer = RegexTokenizer(inputCol="concatText", outputCol="wordConcatText", pattern="\\W")
concatTextStopWordsRemover = StopWordsRemover(inputCol="wordConcatText", outputCol="wordConcatTextNoSW")
concatTextWord2Vec = Word2Vec(vectorSize=50, windowSize=10, minCount=0, inputCol="wordConcatTextNoSW", outputCol="vecConcatText")

keywordLengthTransformer = KeywordLengthTransformer(inputCol="keyword", outputCol="keywordLength")
textNoLinkLengthTransformer = TextNoLinkLengthTransformer(inputCol="textNoLink", outputCol="textNoLinkLength")

catAssembler = VectorAssembler(inputCols=["keywordIndex", "locationIndex", 
                                        "keywordLength", "textNoLinkLength",
                                        "containLink"], outputCol="catFeatures")

catTextAssembler = VectorAssembler(inputCols=["keywordIndex", "locationIndex", 
                                        "keywordLength", "textNoLinkLength",
                                        "containLink", "vecText"], outputCol="catTextFeatures")

catFeatureRobustScaler = RobustScaler(inputCol="catFeatures", outputCol="catFeaturesRobustScaler",
                                      withScaling=True, withCentering=True,
                                      lower=0.25, upper=0.75)

catTextFeatureRobustScaler = RobustScaler(inputCol="catTextFeatures", outputCol="catTextFeaturesRobustScaler",
                                      withScaling=True, withCentering=True,
                                      lower=0.25, upper=0.75)

vecConcatTextRobustScaler = RobustScaler(inputCol="vecConcatText", outputCol="vecConcatTextRobustScaler",
                                      withScaling=True, withCentering=True,
                                      lower=0.25, upper=0.75)

In [394]:
preprocessingPipeline = Pipeline(stages=[textFillNanTransformer,
                                keywordFillNanTransformer,
                                locationFillNanTransformer,
                                containLinkTransformer,
                                textNoLinkTransformer,
                                regexTokenizer,
                                stopWordsRemover,
                                word2Vec,
                                keywordIndexer,
                                locationIndexer,
                                keywordLengthTransformer,
                                textNoLinkLengthTransformer,
                                catAssembler,
                                catTextAssembler,
                                catFeatureRobustScaler,
                                catTextFeatureRobustScaler,
                                concatenateTransformer,
                                concatTextRegexTokenizer,
                                concatTextStopWordsRemover,
                                concatTextWord2Vec,
                                vecConcatTextRobustScaler
                                    ])

## Training

In [395]:
preprocessingModel = preprocessingPipeline.fit(trainData)

trainDataPreprocessed = preprocessingModel.transform(trainData)
testDataPreprocessed = preprocessingModel.transform(testData)

In [396]:
trainSet, validSet = trainDataPreprocessed.randomSplit([0.9, 0.1], seed=12345)

### Pyspark algorithms

In [487]:
featuresCol = "vecConcatText"
labelCol = "target"

In [501]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier, LinearSVC

evaluator = MulticlassClassificationEvaluator(labelCol=labelCol, predictionCol="prediction", metricName="accuracy")

algorithmList = {"LR":   LogisticRegression(featuresCol=featuresCol, labelCol=labelCol,regParam = 0.1, maxIter=50),
                 "DTC":  DecisionTreeClassifier(featuresCol=featuresCol, labelCol=labelCol, maxDepth=5),
                 "RFC":  RandomForestClassifier(featuresCol=featuresCol, labelCol=labelCol, maxDepth=5, numTrees=20),
                 "GBTC": GBTClassifier(featuresCol=featuresCol, labelCol=labelCol, maxIter=50, maxDepth=5, stepSize=0.1),
                 "MPC":  MultilayerPerceptronClassifier(featuresCol=featuresCol, labelCol=labelCol, maxIter=50, layers=[50, 10, 2]),
                 "LSVC": LinearSVC(featuresCol=featuresCol, labelCol=labelCol, maxIter=50, regParam=0.01)
                }

We observe predictions of validation set based on several feature sets:
* "catFeaturesRobustScaler": "keywordIndex", "locationIndex", "keywordLength", "textNoLinkLength", "containLink"
* "catTextFeaturesRobustScaler": "catFeaturesRobustScaler" and "vecText"
* "vecText" only
* "vecConcatText": concatenate "keyword", "location" and "textNoLink", then apply Word2Vec

In [493]:
featuresCols = ["catFeaturesRobustScaler", "catTextFeaturesRobustScaler", 
                "vecText", "vecConcatText"]
algorithmName = "RFC"
for param in featuresCols:
    startTime = time.time()
    algorithm = algorithmList[algorithmName].setFeaturesCol(param)
    prediction = algorithm.fit(trainSet).transform(validSet)
    score = evaluator.evaluate(prediction)
    print(f'Param {param}: {np.round(score,5)} in {np.round(time.time() - startTime, 3)}s')

Param catFeaturesRobustScaler: 0.63477 in 6.302s
Param catTextFeaturesRobustScaler: 0.71563 in 6.822s
Param vecText: 0.70755 in 8.063s
Param vecConcatText: 0.74124 in 6.694s


We observe that concatenating strings in "keyword", "location" and "text" without links produce the best result.

In [503]:
algorithmName = "RFC"
algorithm = algorithmList[algorithmName]
prediction = algorithm.fit(trainSet).transform(validSet)
score = evaluator.evaluate(prediction)
print(score)

0.7412398921832885


In [490]:
import sklearn.metrics as metrics

trueLabel = np.array(validSet.select('target').collect()).squeeze()
predLabel = np.array(prediction.select('prediction').collect()).squeeze()
predProb = np.array(prediction.select('probability').collect()).squeeze()

print(metrics.classification_report(trueLabel, predLabel))

              precision    recall  f1-score   support

           0       0.70      0.84      0.76       411
           1       0.74      0.54      0.63       331

    accuracy                           0.71       742
   macro avg       0.72      0.69      0.69       742
weighted avg       0.72      0.71      0.70       742



### Other algorithms

#### Write data

In [406]:
from os import walk
import re

def getCSVFromPysparkPath(path):
    filenames = next(walk(path), (None, None, []))[2]
    for filename in filenames:
        if re.search('^part', filename):
            return path+'/'+filename

In [494]:
featuresCol = "vecConcatText"
labelCol = "target"

from pyspark.ml.functions import vector_to_array

(trainSet.withColumn("feature", vector_to_array(featuresCol)))\
.select(["target"]+ [F.col("feature")[i] for i in range(50)]).write.mode('overwrite').csv('trainCSV')

(validSet.withColumn("feature", vector_to_array(featuresCol)))\
.select([F.col("feature")[i] for i in range(50)]).write.mode('overwrite').csv('validCSV')

trainPath = getCSVFromPysparkPath("trainCSV")
validPath = getCSVFromPysparkPath("validCSV")

In [495]:
(testDataPreprocessed.withColumn("feature", vector_to_array(featuresCol)))\
.select([F.col("feature")[i] for i in range(50)]).write.mode('overwrite').csv('testCSV')

testPath = getCSVFromPysparkPath("testCSV")

#### Lightgbm

In [496]:
import lightgbm as lgb

In [497]:
trainSetLgb = lgb.Dataset(trainPath)
#validSetLgb = lgb.Dataset(validPath, reference=trainSetLgb)

In [498]:
param = {'num_leaves': 5, 
         'objective': 'binary',
         'metric': 'f1',
         'learning_rate': 0.1,
         'boosting': 'dart',
         'label_column': 0,
        }

In [499]:
num_round=100
bst = lgb.train(param,trainSetLgb, num_round)

In [500]:
import sklearn.metrics as metrics

predProbModel = bst.predict(validPath, num_iteration=bst.best_iteration)

trueLabel = np.array(validSet.select('target').collect()).squeeze()
predLabel = np.array([1 if x >= 0.5 else 0 for x in predProbModel])
predProb = np.array([[1. - x, x] for x in predProbModel])

print(metrics.classification_report(trueLabel, predLabel))

              precision    recall  f1-score   support

           0       0.70      0.85      0.77       411
           1       0.74      0.55      0.63       331

    accuracy                           0.71       742
   macro avg       0.72      0.70      0.70       742
weighted avg       0.72      0.71      0.71       742



## Predict and write file

#### For Pyspark

In [402]:
predictionTest = algorithm.fit(trainDataPreprocessed).transform(testDataPreprocessed)

In [403]:
predLabelTest = np.array(predictionTest.select('prediction').collect()).squeeze()

#### For other algorithms

##### LightGBM

In [436]:
predictionTest = bst.predict(testPath, num_iteration=bst.best_iteration)

In [437]:
predLabelTest = np.array([1 if x >= 0.5 else 0 for x in predictionTest])

#### Upload to kaggle

In [438]:
submission = pd.read_csv('nlp-getting-started/sample_submission.csv')
submission['target'] = submission['target'] + predLabelTest.astype(int)
submission.to_csv('submission.csv', index=False)

In [439]:
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "LightGbm-Concat"

100%|██████████████████████████████████████| 22.2k/22.2k [00:04<00:00, 4.82kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets